In [1]:
import pandas as pd
url = ('D:\Data Analyts course\Python\Project2\Data Wrangling Practise\prices.csv')
price = pd.read_csv(url)
url2 = ('D:\Data Analyts course\Python\Project2\Data Wrangling Practise\sales.csv')
sales = pd.read_csv(url2)

In [2]:
#Đổi các cột thời gian từ dạng Series pandas thành datetime
sales["ordered_at"] = pd.to_datetime(sales["ordered_at"])
price["updated_at"] = pd.to_datetime(price["updated_at"])

In [3]:
#Nối bảng sales và price sao cho thời gian update giá trước hoặc bằng thời gian order để lấy giá mới
merge_tab = pd.merge_asof(sales.sort_values("ordered_at"), price.sort_values("updated_at"), left_on = "ordered_at", right_on = "updated_at", by = "product_id")
merge_tab =  merge_tab [['product_id', 'ordered_at', 'quantity_ordered', 'new_price', 'updated_at']]

In [4]:
#Nối các order chưa khớp giá với bảng price sao cho thời gian update giá sau thời gian order để lấy giá cũ
filtered_df = merge_tab[merge_tab['new_price'].isna()]
sub_tab = pd.merge_asof(filtered_df.sort_values("ordered_at"), price.sort_values("updated_at"),left_on = "ordered_at", right_on = "updated_at", by = "product_id", direction = "forward")
sub_tab = sub_tab [['product_id', 'ordered_at', 'quantity_ordered', 'old_price', 'updated_at_y']]
sub_tab = sub_tab.rename(columns = {'old_price': 'price'})
sub_tab = sub_tab.rename(columns = {'updated_at_y': 'update_at'})

In [6]:
#Xóa các dòng order chưa khớp giá ở merge_tab
tab = merge_tab[merge_tab.new_price.notnull()]
tab = tab.rename(columns = {'new_price':'price'})

In [7]:
#Tạo bảng tổng hợp gồm mã sp, ngày order, số lượng, đơn giá và thành tiền
BTH = pd.concat ([tab, sub_tab], ignore_index = True)
BTH = BTH[['product_id', 'ordered_at', 'quantity_ordered', 'price']]
BTH['total'] = BTH['price'] * BTH['quantity_ordered']
print (BTH)

     product_id          ordered_at  quantity_ordered    price    total
0       3954203 2018-09-11 01:43:00                 1  60000.0  60000.0
1       4085861 2018-09-11 09:30:00                 1  53500.0  53500.0
2       4085861 2018-09-11 11:06:00                 1  53500.0  53500.0
3       3954203 2018-09-11 11:11:00                 1  60000.0  60000.0
4       3954203 2018-09-11 11:11:00                 1  60000.0  60000.0
..          ...                 ...               ...      ...      ...
170     3954203 2018-09-18 21:26:00                 1  57500.0  57500.0
171     3998909 2018-09-18 22:11:00                 1  16500.0  16500.0
172     4085861 2018-09-11 06:26:00                 1  60000.0  60000.0
173     4085861 2018-09-11 06:53:00                 1  60000.0  60000.0
174     4085861 2018-09-11 08:24:00                 1  60000.0  60000.0

[175 rows x 5 columns]


In [21]:
#Bảng tổng doanh thu từng sản phẩm trong tháng 9/2018
BTH.groupby("product_id")["total"].sum()

product_id
64          956000.0
3954203     877500.0
3998909     280500.0
4085861    8247500.0
Name: total, dtype: float64

In [11]:
#Bảng tổng hợp doanh thu theo từng sản phẩm và từng mức giá
BTH.groupby(['product_id', 'price'])['total'].sum()

product_id  price   
64          239000.0     956000.0
3954203     57500.0       57500.0
            60000.0      180000.0
            64000.0      640000.0
3998909     15500.0       15500.0
            16500.0      231000.0
            17000.0       34000.0
4085861     52000.0     1040000.0
            53500.0     2140000.0
            58000.0     2204000.0
            60000.0      180000.0
            62500.0     1812500.0
            67000.0      871000.0
Name: total, dtype: float64